In [1]:
import pandas as pd
import gensim
from gensim.models import FastText

import pandas as pd
import sklearn
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# LIBRARY UNTUK PREPROCESSING
import nltk #import library nltk
from nltk.tokenize import word_tokenize #import word_tokenize for tokenizing text into words 
from nltk.tokenize import sent_tokenize #import sent_tokenize for tokenizing paragraph into sentences
from nltk.stem.porter import PorterStemmer #import Porter Stemmer Algorithm 
from nltk.stem import WordNetLemmatizer #import WordNet lemmatizer 
from nltk.corpus import stopwords #import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #import Indonesian Stemmer
import re,string #import regular expression

import gensim
from gensim.models import KeyedVectors
from tqdm import tqdm

C:\Users\mfaishaldp\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
dta = pd.read_csv('new_Data - Bismillah - Stemming.csv')
dtf = dta

In [3]:
dtf.drop_duplicates(subset='Stemming', inplace = True)
dtf.reset_index(drop=True,inplace=True)

In [4]:
cekDup = dtf.duplicated(subset = 'Stemming')
cekDup.sum()

0

In [5]:
dtf['Label'].value_counts()

-1    7769
 1    6940
 0    5649
Name: Label, dtype: int64

In [6]:
dtf.isna().sum()

Date            0
Text            0
Label           0
Text Clean      1
meaningless     1
Case Folding    1
normalisasi     1
Stopword        1
Stemming        1
tokenizing      0
dtype: int64

In [7]:
dtf.dropna(inplace=True)

In [8]:
dtf.isna().sum()

Date            0
Text            0
Label           0
Text Clean      0
meaningless     0
Case Folding    0
normalisasi     0
Stopword        0
Stemming        0
tokenizing      0
dtype: int64

## Pick Data for Data Test

In [9]:
dtf

,Date,Text,Label,Text Clean,meaningless,Case Folding,normalisasi,Stopword,Stemming,tokenizing
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1,Bagaimana ini gk ada pemberitahuan kpn selesai...,Bagaimana gk ada pemberitahuan kpn selesainy...,bagaimana ini gk ada pemberitahuan kpn selesai...,bagaimana ini tidak ada pemberitahuan kapan se...,bagaimana ini tidak ada pemberitahuan kapan se...,bagaimana ini tidak ada pemberitahuan kapan se...,"['bagaimana', 'ini', 'tidak', 'ada', 'pemberit..."
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0,HOREE awal saldo nol Setelah cek jt gue tarik ...,HOREE awal saldo nol Setelah cek jt gue tarik ...,horee awal saldo nol setelah cek jt gue tarik ...,hore awal saldo nol setelah cek juta saya tari...,hore awal saldo nol setelah cek juta saya tari...,hore awal saldo nol telah cek juta saya tarik ...,"['hore', 'awal', 'saldo', 'nol', 'telah', 'cek..."
2,2019-07-20,Bankerot nih namanya,-1,Bankerot nih namanya,Bankerot nih namanya,bankerot nih namanya,bangkrut ini namanya,bangkrut ini namanya,bangkrut ini nama,"['bangkrut', 'ini', 'nama']"
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1,Alhandulillah Rek Mandiri saya amaann masih se...,Alhandulillah Rek Mandiri amaann kemarin,alhandulillah rek mandiri saya amaann masih se...,alhamdulillah rekening mandiri saya aman masih...,alhamdulillah rekening mandiri saya aman masih...,alhamdulillah rekening mandiri saya aman masih...,"['alhamdulillah', 'rekening', 'mandiri', 'saya..."
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri normal lagi urus...,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri bisa normal lagi u...,mohon info kapan mandiri bisa normal lagi urus...,"['mohon', 'info', 'kapan', 'mandiri', 'bisa', ..."
...,...,...,...,...,...,...,...,...,...,...
20353,2019-04-02,@GdeDyaksa Terima kasih Bpk Dyaksa atas partis...,1,Terima kasih Bpk Dyaksa atas partisipasi yg di...,Terima kasih Bpk Dyaksa atas partisipasi dib...,terima kasih bpk dyaksa atas partisipasi yg di...,terima kasih bapak dyaksa atas partisipasi yan...,terima kasih bapak dyaksa atas partisipasi yan...,terima kasih bapak dyaksa atas partisipasi yan...,"['terima', 'kasih', 'bapak', 'dyaksa', 'atas',..."
20354,2019-04-02,@mohammadiball Terima kasih Bpk Ibal atas part...,1,Terima kasih Bpk Ibal atas partisipasi yg dibe...,Terima kasih Bpk Ibal atas partisipasi diber...,terima kasih bpk ibal atas partisipasi yg dibe...,terima kasih bapak ibal atas partisipasi yang ...,terima kasih bapak ibal atas partisipasi yang ...,terima kasih bapak ibal atas partisipasi yang ...,"['terima', 'kasih', 'bapak', 'ibal', 'atas', '..."
20355,2019-04-02,@JuraganHusin Terima kasih Bpk Husin atas part...,1,Terima kasih Bpk Husin atas partisipasi yg dib...,Terima kasih Bpk Husin atas partisipasi dibe...,terima kasih bpk husin atas partisipasi yg dib...,terima kasih bapak husin atas partisipasi yang...,terima kasih bapak husin atas partisipasi yang...,terima kasih bapak husin atas partisipasi yang...,"['terima', 'kasih', 'bapak', 'husin', 'atas', ..."
20356,2019-04-02,@vill4nk Terima kasih Bpk/Ibu atas partisipasi...,1,nk Terima kasih Bpk Ibu atas partisipasi yg di...,nk Terima kasih Bpk Ibu atas partisipasi dib...,nk terima kasih bpk ibu atas partisipasi yg di...,natural killer terima kasih bapak ibu atas par...,natural killer terima kasih bapak ibu atas par...,natural killer terima kasih bapak ibu atas par...,"['natural', 'killer', 'terima', 'kasih', 'bapa..."


In [10]:
dtf['Date'] = pd.to_datetime(dtf['Date'])

In [11]:
dtf.sort_values(by='Date', inplace=True)

In [12]:
dtf.reset_index(drop=True, inplace=True)

In [13]:
dtf

,Date,Text,Label,Text Clean,meaningless,Case Folding,normalisasi,Stopword,Stemming,tokenizing
0,2019-04-01,"@muliasm @bankmandiri Selamat pagi Ibu Mulia, ...",1,Selamat pagi Ibu Mulia berikut cara update sal...,Selamat pagi Ibu Mulia berikut cara update sal...,selamat pagi ibu mulia berikut cara update sal...,selamat pagi ibu mulia berikut cara pembaruan ...,selamat pagi ibu mulia berikut cara pembaruan ...,selamat pagi ibu mulia ikut cara baru saldo em...,"['selamat', 'pagi', 'ibu', 'mulia', 'ikut', 'c..."
1,2019-04-01,@zhiepman @ovo_id @bankmandiri Hallo Bpk Candr...,1,id Hallo Bpk Candra Mohon maaf ats ketidaknyam...,id Hallo Bpk Candra Mohon maaf ats ketidaknyam...,id hallo bpk candra mohon maaf ats ketidaknyam...,id halo bapak candra mohon maaf atas ketidakny...,id halo bapak candra mohon maaf atas ketidakny...,id halo bapak candra mohon maaf atas ketidakny...,"['id', 'halo', 'bapak', 'candra', 'mohon', 'ma..."
2,2019-04-01,Saya top up @ovo_id pakai mandiri mobile @mand...,1,Saya top up id pakai mandiri mobile belum juga...,Saya top up id pakai mandiri mobile masuk ...,saya top up id pakai mandiri mobile belum juga...,saya top up id pakai mandiri mobile belum juga...,saya top up id pakai mandiri mobile belum juga...,saya top up id pakai mandiri mobile belum juga...,"['saya', 'top', 'up', 'id', 'pakai', 'mandiri'..."
3,2019-04-01,"@getavs Baik Ibu Disarana, apabila Ibu sudah m...",1,Baik Ibu Disarana apabila Ibu sudah mengirimka...,Baik Ibu Disarana apabila Ibu mengirimkan pe...,baik ibu disarana apabila ibu sudah mengirimka...,baik ibu disarana apabila ibu sudah mengirimka...,baik ibu disarana apabila ibu sudah mengirimka...,baik ibu sarana apabila ibu sudah kirim pesan ...,"['baik', 'ibu', 'sarana', 'apabila', 'ibu', 's..."
4,2019-04-01,Kolaborasi BSM dengan Hijup untuk Jadi Mitra L...,1,Kolaborasi BSM dengan Hijup untuk Jadi Mitra L...,Kolaborasi BSM Hijup Jadi Mitra Layanan Pe...,kolaborasi bsm dengan hijup untuk jadi mitra l...,kolaborasi bsm dengan hijup untuk jadi mitra l...,kolaborasi bsm dengan hijup untuk jadi mitra l...,kolaborasi bsm dengan hijup untuk jadi mitra l...,"['kolaborasi', 'bsm', 'dengan', 'hijup', 'untu..."
...,...,...,...,...,...,...,...,...,...,...
20352,2019-12-30,Nipu kurang riset adalah pagi ini tiba-tiba ad...,0,Nipu kurang riset adalah pagi ini tiba ada tel...,Nipu kurang riset pagi tiba ada telepon bi...,nipu kurang riset adalah pagi ini tiba ada tel...,menipu kurang riset adalah pagi ini tiba ada t...,menipu kurang riset adalah pagi ini tiba ada t...,tipu kurang riset adalah pagi ini tiba ada tel...,"['tipu', 'kurang', 'riset', 'adalah', 'pagi', ..."
20353,2019-12-30,@mandiricare saya abdul hamid nasabah bank man...,-1,saya abdul hamid nasabah bank mandiri hari ini...,abdul hamid nasabah bank mandiri hari ma...,saya abdul hamid nasabah bank mandiri hari ini...,saya abdul hamid nasabah bank mandiri hari ini...,saya abdul hamid nasabah bank mandiri hari ini...,saya abdul hamid nasabah bank mandiri hari ini...,"['saya', 'abdul', 'hamid', 'nasabah', 'bank', ..."
20354,2019-12-31,"@mandiricare hpku ada NFCnya, ngisi saldo e mo...",1,hpku ada NFCnya ngisi saldo e money di m banki...,hpku ada NFCnya ngisi saldo e money m bankin...,hpku ada nfcnya ngisi saldo e money di m banki...,hpku ada nfcnya mengisi saldo e uang di sama b...,hpku ada nfcnya mengisi saldo uang di sama ban...,hpku ada nfcnya isi saldo uang di sama banking...,"['hpku', 'ada', 'nfcnya', 'isi', 'saldo', 'uan..."
20355,2019-12-31,"@mandiricare selamat pagi, saya transfer uang ...",-1,selamat pagi saya transfer uang dari bank mand...,selamat pagi transfer uang bank mandiri ...,selamat pagi saya transfer uang dari bank mand...,selamat pagi saya transfer uang dari bank mand...,selamat pagi saya transfer uang dari bank mand...,selamat pagi saya transfer uang dari bank mand...,"['selamat', 'pagi', 'saya', 'transfer', 'uang'..."


In [14]:
# april
# april = dtf.loc[0:5907,['Date','Stemming','Label']].reset_index(drop=True)
# mei
# mei = dtf.loc[5908:6207,['Date','Stemming','Label']].reset_index(drop=True)
# juni
# juni = dtf.loc[6208:6551,['Date','Stemming','Label']].reset_index(drop=True)
# Juli
# juli = dtf.loc[6552:18502,['Date','Stemming','Label']].reset_index(drop=True)
# Agustus
# agustus = dtf.loc[18503:19227,['Date','Stemming','Label']].reset_index(drop=True)
# September
# september = dtf.loc[19228:19698,['Date','Stemming','Label']].reset_index(drop=True)
# Oktober
# oktober = dtf.loc[19699:20161,['Date','Stemming','Label']].reset_index(drop=True)
# November
# november = dtf.loc[20162:20254,['Date','Stemming','Label']].reset_index(drop=True)
# Desember
# desember = dtf.loc[20255:20356,['Date','Stemming','Label']].reset_index(drop=True)

apriljuli = dtf.loc[0:18502,['Date','Stemming','Label']].reset_index(drop=True)

In [15]:
apriljuli

,Date,Stemming,Label
0,2019-04-01,selamat pagi ibu mulia ikut cara baru saldo em...,1
1,2019-04-01,id halo bapak candra mohon maaf atas ketidakny...,1
2,2019-04-01,saya top up id pakai mandiri mobile belum juga...,1
3,2019-04-01,baik ibu sarana apabila ibu sudah kirim pesan ...,1
4,2019-04-01,kolaborasi bsm dengan hijup untuk jadi mitra l...,1
...,...,...,...
18498,2019-07-31,jadi eror saldo bank mandiri tempo hari ada ju...,-1
18499,2019-07-31,yang hebat itu bmri fatal eror hilang saldo na...,-1
18500,2019-07-31,tanda banyak ketidakberesan badan usaha milik ...,-1
18501,2019-07-31,gilee ini bmri suka banget sama begini,0


## CEK EKSPANSI

In [68]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dtf['Stemming'], dtf['Label'], test_size=0.10, shuffle = True, random_state=12)

In [69]:
# x_test = april['Stemming']
# x_test = mei['Stemming']
# x_test = juni['Stemming']
# x_test = juli['Stemming']
# x_test = agustus['Stemming']
# x_test = september['Stemming']
# x_test = oktober['Stemming']
# x_test = november['Stemming']
# x_test = desember['Stemming']
x_test = apriljuli['Stemming']

# y_test = april['Label']
# y_test = mei['Label']
# y_test = juni['Label']
# y_test = juli['Label']
# y_test = agustus['Label']
# y_test = september['Label']
# y_test = oktober['Label']
# y_test = november['Label']
# y_test = desember['Label']
y_test = apriljuli['Label']

In [70]:
#mencoba tanpa max feature
vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features = 7000)

In [71]:
# df_tfidf = vectorizer.fit_transform(dtf['Stemming'].astype(str))

x_train_tfidf = vectorizer.fit_transform(x_train.astype(str))
x_test_tfidf = vectorizer.transform(x_test.astype(str))

In [72]:
print(x_train_tfidf.shape)
print(x_test_tfidf.shape)

(18321, 7000)
(18503, 7000)


In [23]:
# Load Model trained data actual corpus (sg = 0)
# model_ef = FastText.load('modelft-26jan-Dataactual')

In [25]:
# Load Model trained gab data actual dan pretrained corpus (sg = 0)
# model_ef = FastText.load('modelft-14jan-actualpretrained')

In [26]:
# Load Model Ekspansi Fitur
model_ef = gensim.models.KeyedVectors.load_word2vec_format('cc.id.300.vec.gz')

In [73]:
def feature_expansion(df, feature):
    for col in tqdm(df.columns): #loop per kolom
        try: 
            sim_word = model_ef.similar_by_word(col, topn = 1) #mencari Similarity
        except:
            sim_word = []
        if sim_word != []: #kalo similarity-nya tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [74]:
feature_ef = vectorizer.get_feature_names()

In [75]:
# Membuat dataframe data test & train

df_x_train_tfidf = pd.DataFrame(x_train_tfidf.todense(), columns = feature_ef)
df_x_test_tfidf = pd.DataFrame(x_test_tfidf.todense(), columns = feature_ef)

In [76]:
# Membuat Model Logistic Regression
logreg = LogisticRegression()

In [77]:
df_x_train_ef = feature_expansion(df_x_train_tfidf, feature_ef)

100%|██████████| 7000/7000 [14:53<00:00,  7.83it/s]


In [78]:
df_x_test_ef = feature_expansion(df_x_test_tfidf, feature_ef)

100%|██████████| 7000/7000 [16:48<00:00,  6.94it/s]


In [79]:
ef_params = {'penalty' : ['l2'],
            'C' : [1],
            'solver' : ['saga'],
            'multi_class' : ['ovr'],
            'max_iter' : [10000]}

ef_clf = GridSearchCV(logreg, ef_params, refit = True, verbose = 3, cv=5)
ef_clf.fit(df_x_train_ef,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END C=1, max_iter=10000, multi_class=ovr, penalty=l2, solver=saga; total time= 2.1min
[CV 2/5] END C=1, max_iter=10000, multi_class=ovr, penalty=l2, solver=saga; total time= 2.1min
[CV 3/5] END C=1, max_iter=10000, multi_class=ovr, penalty=l2, solver=saga; total time= 1.7min
[CV 4/5] END C=1, max_iter=10000, multi_class=ovr, penalty=l2, solver=saga; total time= 2.1min
[CV 5/5] END C=1, max_iter=10000, multi_class=ovr, penalty=l2, solver=saga; total time= 1.9min


GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [1], 'max_iter': [10000], 'multi_class': ['ovr'],
                         'penalty': ['l2'], 'solver': ['saga']},
             verbose=3)

In [80]:
y_predi_ef = ef_clf.predict(df_x_test_ef)

In [81]:
print(classification_report(y_test, y_predi_ef))

              precision    recall  f1-score   support

          -1       0.75      0.88      0.81      7029
           0       0.69      0.59      0.63      4758
           1       0.86      0.80      0.83      6716

    accuracy                           0.78     18503
   macro avg       0.77      0.76      0.76     18503
weighted avg       0.78      0.78      0.77     18503



In [82]:
accuracy_score(y_test, y_predi_ef)

0.7751175485056477

## SATUKAN LABEL PREDIKSI DENGAN DF

In [83]:
y_predi_ef

array([ 1,  1,  1, ..., -1, -1,  0], dtype=int64)

In [84]:
apriljuli

,Date,Stemming,Label
0,2019-04-01,selamat pagi ibu mulia ikut cara baru saldo em...,1
1,2019-04-01,id halo bapak candra mohon maaf atas ketidakny...,1
2,2019-04-01,saya top up id pakai mandiri mobile belum juga...,1
3,2019-04-01,baik ibu sarana apabila ibu sudah kirim pesan ...,1
4,2019-04-01,kolaborasi bsm dengan hijup untuk jadi mitra l...,1
...,...,...,...
18498,2019-07-31,jadi eror saldo bank mandiri tempo hari ada ju...,-1
18499,2019-07-31,yang hebat itu bmri fatal eror hilang saldo na...,-1
18500,2019-07-31,tanda banyak ketidakberesan badan usaha milik ...,-1
18501,2019-07-31,gilee ini bmri suka banget sama begini,0


In [85]:
apriljuli_pred = apriljuli
apriljuli_pred['Label Predict'] = y_predi_ef.tolist()

In [86]:
apriljuli_pred

,Date,Stemming,Label,Label Predict
0,2019-04-01,selamat pagi ibu mulia ikut cara baru saldo em...,1,1
1,2019-04-01,id halo bapak candra mohon maaf atas ketidakny...,1,1
2,2019-04-01,saya top up id pakai mandiri mobile belum juga...,1,1
3,2019-04-01,baik ibu sarana apabila ibu sudah kirim pesan ...,1,1
4,2019-04-01,kolaborasi bsm dengan hijup untuk jadi mitra l...,1,1
...,...,...,...,...
18498,2019-07-31,jadi eror saldo bank mandiri tempo hari ada ju...,-1,-1
18499,2019-07-31,yang hebat itu bmri fatal eror hilang saldo na...,-1,-1
18500,2019-07-31,tanda banyak ketidakberesan badan usaha milik ...,-1,-1
18501,2019-07-31,gilee ini bmri suka banget sama begini,0,-1


In [87]:
# apriljuli_pred.to_csv('apriljuli_pred_3feb.csv', header = True, index = False)